In [43]:
import pandas as pd 
import numpy as np
import pyecharts
import plotly.express as px # visualization
import pandas_datareader as pdr # access fred
from datetime import date
from fredapi import Fred
import requests
import openpyxl
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta
import re

from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK

from pyecharts import options as opts
from pyecharts.charts import Bar, Line

In [412]:
# Load orginal dataset

index_data = pd.read_excel('/Users/jackyliu/Desktop/python/pmi/pmi_level.xlsx')
glance = pd.read_excel('/Users/jackyliu/Desktop/python/pmi/glancetable.xlsx')
comments = pd.read_excel('/Users/jackyliu/Desktop/python/pmi/comments.xlsx')
sector_ranking = pd.read_excel('/Users/jackyliu/Desktop/python/pmi/sector_ranking.xlsx')

In [219]:
# Copy and paste latest month data url

url = 'https://www.prnewswire.com/news-releases/manufacturing-pmi-at-46-7-october-2023-manufacturing-ism-report-on-business-301973296.html'

response = requests.get(url)

### Extract Data from URL ###

#### 1. Get comments from respondents ####

In [220]:
# Extract What Respondents Say

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')   
    section = soup.find('section', class_='release-body container')

    if section:
        main_content = section.find('div', class_='col-lg-10 col-lg-offset-1')

        if main_content:
            ul_element = main_content.find('ul', type='disc')
            if ul_element:
                list_items = ul_element.find_all('li')
                if list_items:
                    # Create a list to store the text of each list item
                    items_list = [item.text.strip() for item in list_items]

                    # Create a DataFrame from the list
                    respondent_comment = pd.DataFrame(items_list, columns=['WHAT RESPONDENTS ARE SAYING'])
        else:
            print("Main content not found within the section.")
    else:
        print("Section with class 'xxxx' not found.")

else:
    print("Failed to retrieve the page")

#### 2. Get Monthly Level Data ####

In [221]:
# Extract monthly index level

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')   
    section = soup.find('section', class_='release-body container')

    if section:
        table_div = section.find('div', class_='table-responsive')

        if table_div:
        # Find the table within the div
            table = table_div.find('table')

            if table:
                # Read the table into a DataFrame
                data_table = pd.read_html(str(table))[0]
            else:
                print("Table not found within the div with class 'table-responsive'")
        else:
            print("Div with class 'table-responsive' not found.")
    else:
        print("Failed to retrieve the page")

/var/folders/j9/4m63qjgj5g350s7xnpn3kzs40000gp/T/ipykernel_2179/2319469250.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data_table = pd.read_html(str(table))[0]


#### 3. Get Commodity Data ####

In [222]:
# Extract Commodity up/down/short supply

commodity_up = soup.select_one('#main > article > section > div > div > p:nth-child(16)')
commodity_down = soup.select_one('#main > article > section > div > div > p:nth-child(17)')
commodity_short = soup.select_one('#main > article > section > div > div > p:nth-child(18)')

commodity_tags = [commodity_up,commodity_down,commodity_short]
commoditiy_direction = []
for i, tag in enumerate(commodity_tags):
    if tag:
        direction = tag.get_text(separator=' ').strip()  # Get all the text within the <p> tag
        commoditiy_direction.append(direction)
    else:
        print(f"The specified <p> tag {i+1} was not found.")

# Create a DataFrame with one column and three rows from the list of contents
commodity_comments = pd.DataFrame(commoditiy_direction, columns=['content'])

In [285]:
substring_patterns = ['Commodities Up in Price','Commodities Down in Price','Commodities in']

def split_based_on_pattern(content, pattern):
    if pattern in content:
        parts = content.split(pattern, 1)
        return [pattern.strip(), parts[1].strip()]
    else:
        return [content.strip(), '']

# Split each row based on the defined patterns
split_data = []
for i, row in commodity_comments.iterrows():
    split_data.append(split_based_on_pattern(row['content'], substring_patterns[i]))

# Create a new DataFrame from the split data
commodity_df = pd.DataFrame(split_data, columns=['pattern', 'rest'])

# Display the split DataFrame
commodity_df

,pattern,rest
0,Commodities Up in Price,Crude Oil (3); Electronic Components (2); Labor — Professional Services; Labor — Temporary (2); Natural Gas (4); Plastic Resins* (2); Polypropylen...
1,Commodities Down in Price,Aluminum (5); Caustic Soda (4); Copper Based Products; Corrugate Boxes (3); Packaging; Plastic Resins* (17); Road Freight*; Steel* (7); Steel — Ho...
2,Commodities in,Short Supply Electrical Components (37); Electrical Equipment; and Electronic Components (35).


#### 4. Get Industry Ranking #### 

In [264]:
# Extract Industry Rankings

pmi = soup.select_one('#main > article > section > div > div > p:nth-child(10)')
new_order = soup.select_one('#main > article > section > div > div > p:nth-child(26)')
production = soup.select_one('#main > article > section > div > div > p:nth-child(29)')
employment= soup.select_one('#main > article > section > div > div > p:nth-child(32)')
supplier_deliveries = soup.select_one('#main > article > section > div > div > p:nth-child(35)')
inventories = soup.select_one('#main > article > section > div > div > p:nth-child(38)')
customers_inv = soup.select_one('#main > article > section > div > div > p:nth-child(41)')
prices = soup.select_one('#main > article > section > div > div > p:nth-child(44)')
backlog_orders = soup.select_one('#main > article > section > div > div > p:nth-child(47)')
new_export_orders = soup.select_one('#main > article > section > div > div > p:nth-child(50)')
imports = soup.select_one('#main > article > section > div > div > p:nth-child(53)')

industry_ranking = [pmi,new_order,production,employment,supplier_deliveries,inventories,customers_inv,prices,backlog_orders,new_export_orders,imports]

rankings = []
for i, tag in enumerate(industry_ranking):
    if tag:
        ranking = tag.get_text().strip()  # Get all the text within the <p> tag
        rankings.append(ranking)
    else:
        print(f"The specified <p> tag {i+1} was not found.")

# Create a DataFrame with one column and three rows from the list of contents
ranking_df = pd.DataFrame(rankings, columns=['Ranking'])
ranking_df['Ranking'] = ranking_df['Ranking'].str.strip()


In [265]:
industry_list = ['PMI','New Order','Production','Employment','Supplier Deliveries','Inventories','Customers_inv','Prices','Backlog Orders','New Export Orders','Imports']
ranking_df['Industry'] = industry_list
col_oder = ['Industry','Ranking']
ranking_df = ranking_df[col_oder]

In [266]:
# Data pre-check
pre_check_dfs = [respondent_comment, data_table, commodity_df, ranking_df]
respondent_comment

,WHAT RESPONDENTS ARE SAYING
0,"""Markets remain tough, and we have focused more resources on sales and marketing to drive greater sales and new market penetration with our device..."
1,"""Economy absolutely slowing down. Less optimism regarding the first quarter of 2024."" [Chemical Products]"
2,"""Backlog is starting to dip a bit. We're hearing of cutbacks in 2024 ordering, but it's still very strong compared to historical averages."" [Trans..."
3,"""Markets appear to have slightly slowed. Certain commodities remain high."" [Food, Beverage & Tobacco Products]"
4,"""Seeing a slowdown on bookings, and our backlog is down to five days from 15 weeks earlier this year."" [Machinery]"
5,"""A slow fourth quarter, and we're clearly in a mild industry recession. However, demand is down less than 5 percent, and customer confidence of a ..."
6,"""Business is decent — not great, but steady and solid. We are meeting our sales and margin goals, but it's definitely hard to guess the future."" [..."
7,"""Commercial constructions continue to remain ahead of 2022. We have some concern over 2024 regarding inflation, as well as gas and oil pricing pot..."
8,"""Demand for raw materials/chemicals appears to be stable heading into the fourth quarter."" [Petroleum & Coal Products]"
9,"""Orders continue to increase in some sectors. Construction industry-related products/orders are slowing down."" [Plastics & Rubber Products]"


In [267]:
data_table

,0,1,2,3,4,5,6
0,MANUFACTURING AT A GLANCE October 2023,MANUFACTURING AT A GLANCE October 2023,MANUFACTURING AT A GLANCE October 2023,MANUFACTURING AT A GLANCE October 2023,MANUFACTURING AT A GLANCE October 2023,MANUFACTURING AT A GLANCE October 2023,MANUFACTURING AT A GLANCE October 2023
1,Index,Series Index Oct,Series Index Sep,Percentage Point Change,Direction,Rate of Change,Trend* (Months)
2,Manufacturing PMI®,46.7,49.0,-2.3,Contracting,Faster,12
3,New Orders,45.5,49.2,-3.7,Contracting,Faster,14
4,Production,50.4,52.5,-2.1,Growing,Slower,2
5,Employment,46.8,51.2,-4.4,Contracting,From Growing,1
6,Supplier Deliveries,47.7,46.4,+1.3,Faster,Slower,13
7,Inventories,43.3,45.8,-2.5,Contracting,Faster,8
8,Customers' Inventories,48.6,47.1,+1.5,Too Low,Slower,5
9,Prices,45.1,43.8,+1.3,Decreasing,Slower,6


In [268]:
commodity_df

,pattern,rest
0,Commodities Up in Price,Crude Oil (3); Electronic Components (2); Labor — Professional Services; Labor — Temporary (2); Natural Gas (4); Plastic Resins* (2); Polypropylen...
1,Commodities Down in Price,Aluminum (5); Caustic Soda (4); Copper Based Products; Corrugate Boxes (3); Packaging; Plastic Resins* (17); Road Freight*; Steel* (7); Steel — Ho...
2,Commodities in,Short Supply Electrical Components (37); Electrical Equipment; and Electronic Components (35).


In [269]:
ranking_df

,Industry,Ranking
0,PMI,"The two manufacturing industries that reported growth in October are: Food, Beverage & Tobacco Products; and Plastics & Rubber Products. The 13 in..."
1,New Order,The three manufacturing industries that reported growth in new orders in October are: Plastics & Rubber Products; Primary Metals; and Transportati...
2,Production,"The four industries reporting growth in production during the month of October are: Paper Products; Plastics & Rubber Products; Food, Beverage & T..."
3,Employment,"Of 18 manufacturing industries, four reported employment growth in October: Nonmetallic Mineral Products; Machinery; Transportation Equipment; and..."
4,Supplier Deliveries,"The two manufacturing industries reporting slower supplier deliveries in October are: Wood Products; and Food, Beverage & Tobacco Products. The se..."
5,Inventories,"Of 18 manufacturing industries, the only one reporting higher inventories in October is Food, Beverage & Tobacco Products. The 12 industries repor..."
6,Customers_inv,The three industries reporting customers' inventories as too high in October are: Textile Mills; Plastics & Rubber Products; and Computer & Electr...
7,Prices,"In October, the two industries that reported paying increased prices for raw materials are: Nonmetallic Mineral Products; and Plastics & Rubber Pr..."
8,Backlog Orders,"The three industries reporting growth in order backlogs in October are: Primary Metals; Food, Beverage & Tobacco Products; and Transportation Equi..."
9,New Export Orders,The six industries reporting growth in new export orders in October — in the following order — are: Nonmetallic Mineral Products; Paper Products; ...


### Working on Data Clenaing ### 

##### 1. PMI Index Level #####

In [270]:
# variables = data_table, respondent_comment, commodity_df, ranking_df
# 1. Working on data_table

new_table = data_table.iloc[1:]
header = new_table.iloc[0]
final_table = new_table.iloc[1:]
final_table.columns = header

In [271]:
new_month_level = final_table.iloc[:,1]
new_month_level = new_month_level[:-2]
new_month_level = new_month_level.to_frame().T

In [272]:
latest_date = index_data['Date'].max()
new_date = latest_date + relativedelta(months=1)
new_date_str = new_date.strftime('%Y-%m-%d')
new_month_level["Date"] = new_date
new_month_level = new_month_level[['Date'] + [col for col in new_month_level if col != 'Date']]
new_month_level.columns = index_data.columns

updated_index_data = pd.concat([index_data, new_month_level], ignore_index=True)

##### 2. What Respondents Saying #####

In [273]:
#pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 150)
respondent_comment

,WHAT RESPONDENTS ARE SAYING
0,"""Markets remain tough, and we have focused more resources on sales and marketing to drive greater sales and new market penetration with our device..."
1,"""Economy absolutely slowing down. Less optimism regarding the first quarter of 2024."" [Chemical Products]"
2,"""Backlog is starting to dip a bit. We're hearing of cutbacks in 2024 ordering, but it's still very strong compared to historical averages."" [Trans..."
3,"""Markets appear to have slightly slowed. Certain commodities remain high."" [Food, Beverage & Tobacco Products]"
4,"""Seeing a slowdown on bookings, and our backlog is down to five days from 15 weeks earlier this year."" [Machinery]"
5,"""A slow fourth quarter, and we're clearly in a mild industry recession. However, demand is down less than 5 percent, and customer confidence of a ..."
6,"""Business is decent — not great, but steady and solid. We are meeting our sales and margin goals, but it's definitely hard to guess the future."" [..."
7,"""Commercial constructions continue to remain ahead of 2022. We have some concern over 2024 regarding inflation, as well as gas and oil pricing pot..."
8,"""Demand for raw materials/chemicals appears to be stable heading into the fourth quarter."" [Petroleum & Coal Products]"
9,"""Orders continue to increase in some sectors. Construction industry-related products/orders are slowing down."" [Plastics & Rubber Products]"


In [274]:
updated_comments = respondent_comment['WHAT RESPONDENTS ARE SAYING'].str.split('[', expand=True)
updated_comments.columns = [new_date_str, 'Industry']
updated_comments['Industry'] = updated_comments['Industry'].str.replace(']', '', regex=False)
updated_comments[new_date_str] = updated_comments[new_date_str].str.replace('"', '', regex=False)
updated_comments = updated_comments.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

col_order = ['Industry',new_date_str]
updated_comments = updated_comments[col_order]

In [275]:
industry_col1 = comments['Industry']
industry_col2 = updated_comments['Industry']

merged_df = pd.concat([industry_col1, industry_col2], ignore_index=True)
merged_series = merged_df.squeeze().drop_duplicates()
merge_comment = merged_series.to_frame(name='Industry')

In [276]:
merge_final_df1 = pd.merge(merge_comment, comments, on='Industry', how='left')
final_comment = pd.merge(merge_final_df1, updated_comments, on='Industry', how='left')

##### 3. Commodity Product Pricing #####

In [332]:
commodity_df

commodity_dfs = []

for index, row in commodity_df.iterrows():
    # Create a new DataFrame for the current row
    row_df = pd.DataFrame([row])
    # Append the new DataFrame to the list
    commodity_dfs.append(row_df)
    

In [388]:
commodity_up_df = commodity_dfs[0]
commodity_up_df_split = commodity_up_df['rest'].str.split(';', expand = True).T
commodity_up_header = ['Commodities Up in Price']
commodity_up_df_split.columns = commodity_up_header


commodity_down_df = commodity_dfs[1]
commodity_down_df_split = commodity_down_df['rest'].str.split(';', expand = True).T
commodity_down_header = ['Commodities Down in Price']
commodity_down_df_split.columns = commodity_down_header

commodity_short_df = commodity_dfs[2]
commodity_short_df_split = commodity_short_df['rest'].str.split(';', expand = True).T
commodity_short_header = ['Commodities in Short Supply']
commodity_short_df_split.columns = commodity_short_header

commodity_up_df

In [389]:
commodity_up_df_split[['Commodity Price Up', 'Number of Month']] = commodity_up_df_split['Commodities Up in Price'].str.split("(", n=1, expand=True)
commodity_up_df_split['Commodity Price Up'] = commodity_up_df_split['Commodity Price Up'].str.strip().str.replace(r'\band\b|\*|\.', '', regex=True)
commodity_up_df_split['Number of Month'] = commodity_up_df_split['Number of Month'].str.rstrip(")").fillna('1')
commodity_up_df_split = commodity_up_df_split.drop(columns=['Commodities Up in Price'])

commodity_down_df

In [390]:
commodity_down_df_split[['Commodity Price Down', 'Number of Month']] = commodity_down_df_split['Commodities Down in Price'].str.split("(", n=1, expand=True)
commodity_down_df_split['Commodity Price Down'] = commodity_down_df_split['Commodity Price Down'].str.strip().str.replace(r'\band\b|\*|\.', '', regex=True)
commodity_down_df_split['Number of Month'] = commodity_down_df_split['Number of Month'].str.rstrip(")").fillna('1')
commodity_down_df_split = commodity_down_df_split.drop(columns=['Commodities Down in Price'])

Commodity in short supply

In [391]:
commodity_short_df_split[['Commodities: Short Supply', 'Number of Month']] = commodity_short_df_split['Commodities in Short Supply'].str.split("(", n=1, expand=True)
commodity_short_df_split['Commodities: Short Supply'] = commodity_short_df_split['Commodities: Short Supply'].str.strip().str.replace(r'\band\b|\*|\.', '', regex=True)
commodity_short_df_split['Number of Month'] = commodity_short_df_split['Number of Month'].str.rstrip(")").str.rstrip(").").fillna('1')
commodity_short_df_split = commodity_short_df_split.drop(columns=['Commodities in Short Supply'])
commodity_short_df_split['Commodities: Short Supply'] = commodity_short_df_split['Commodities: Short Supply'].str.replace('Short Supply ', '', regex=True)

Commodity Data Final Output

In [393]:
commodity_short_df_split = commodity_short_df_split.sort_values(['Number of Month'])
commodity_up_df_split = commodity_up_df_split.sort_values(['Number of Month'])
commodity_down_df_split = commodity_down_df_split.sort_values(['Number of Month'])

##### 4. Industry Ranking #####

In [413]:
ranking_df['Ranking'] = ranking_df['Ranking'].str.strip().str.replace('—','',regex = False).str.replace(',','',regex = False)
ranking_df_split = ranking_df['Ranking'].str.split('.', expand = True)
ranking_df_split.columns = ['Industry Up','Industry Down','No Change','None']

In [414]:
ranking_up_df = pd.concat([ranking_df['Industry'], ranking_df_split['Industry Up']], axis=1)
ranking_down_df = pd.concat([ranking_df['Industry'], ranking_df_split['Industry Down']], axis=1)

In [415]:
ranking_up = ranking_up_df['Industry Up'].str.split(':', expand=True)
ranking_down = ranking_down_df['Industry Down'].str.split(':', expand=True)

col_name = ['split_col','ranking_col']
ranking_up.columns = col_name
ranking_down.columns = col_name

In [416]:
filter_is_up = ranking_up['split_col'].str.split('is ', expand=True)
filter_is_down = ranking_down['split_col'].str.split('is ', expand=True)

In [417]:
filter_is_up = pd.concat([ranking_df['Industry'], filter_is_up], axis=1)
filter_is_down = pd.concat([ranking_df['Industry'], filter_is_down], axis=1)

In [418]:
def select_columns(df):
    if len(df.columns) >= 3:  # Check if DataFrame has at least 3 columns
        return df.iloc[:, [0, 2]]  # Select first and third columns
    else:
        return df.iloc[:, [0]]  # Select only the first column

filter_is_up_df = select_columns(filter_is_up)
filter_is_down_df = select_columns(filter_is_down)

In [419]:
ranking_up_final = pd.concat([ranking_df['Industry'], ranking_up['ranking_col']], axis=1)
ranking_down_final = pd.concat([ranking_df['Industry'], ranking_down['ranking_col']], axis=1)

In [420]:
ranking_up_final = pd.merge(ranking_up_final,filter_is_up_df, on= 'Industry')
ranking_down_final = pd.merge(ranking_down_final,filter_is_down_df, on= 'Industry')

In [421]:
def merge_columns(df):
    if len(df.columns) >= 3:
        df['Combined'] = df.iloc[:, 1].fillna(df.iloc[:, 2])
        df.drop(['ranking_col', 1], axis=1, inplace=True)
        return df
    else:
        df.rename(columns={'ranking_col': 'Combined'}, inplace=True)
        return df

ranking_up_final = merge_columns(ranking_up_final)
ranking_down_final = merge_columns(ranking_down_final)

In [422]:
ranking_up_final = ranking_up_final['Combined'].str.split(';', expand=True)
ranking_down_final = ranking_down_final['Combined'].str.split(';', expand=True)

In [423]:
ranking_up_final = ranking_up_final.map(lambda x: x.replace('and', '').strip() if isinstance(x, str) else x)
ranking_down_final = ranking_down_final.map(lambda x: x.replace('and', '').strip() if isinstance(x, str) else x)

In [424]:
ranking_up_final = pd.concat([ranking_df['Industry'], ranking_up_final], axis=1)
ranking_down_final = pd.concat([ranking_df['Industry'], ranking_down_final], axis=1)

In [425]:
rakning_up_column_order = [ranking_up_final.columns[0]] + list(range(1, len(ranking_up_final.columns)))
rakning_down_column_order = [ranking_down_final.columns[0]] + list(range(-1, -len(ranking_down_final.columns), -1))

ranking_up_final.columns = rakning_up_column_order
ranking_down_final.columns = rakning_down_column_order

In [426]:
ranking_up_df = pd.melt(ranking_up_final, id_vars=['Industry'],var_name='myVarname', value_name='myValname')
ranking_down_df = pd.melt(ranking_down_final, id_vars=['Industry'],var_name='myVarname', value_name='myValname')

In [427]:
industry_ranking_final = pd.concat([ranking_up_df, ranking_down_df],ignore_index=True)

industry_ranking_final = industry_ranking_final.rename(columns={'Industry': 'Index','myVarname': new_date_str, 'myValname': 'Industry'})

new_col_order = ['Index','Industry',new_date_str]
industry_ranking_final = industry_ranking_final[new_col_order]


In [428]:
desired_order = ranking_df['Industry'].to_list()

industry_ranking_final['Index'] = pd.Categorical(industry_ranking_final['Index'], categories=desired_order, ordered=True)
industry_ranking_final = industry_ranking_final.sort_values(['Index',new_date_str])
latest_industry_ranking = industry_ranking_final.dropna(subset=['Industry'])

sector_ranking_final = pd.merge(sector_ranking, latest_industry_ranking, on=['Index', 'Industry'], how='left')

# Final Step - Consolidation & Exporting  #

In [429]:
# 1. Output below does not require data overrite 

final_table.to_excel('/Users/jackyliu/Desktop/python/pmi/monthly_table.xlsx', index=False)
commodity_short_df_split.to_excel('/Users/jackyliu/Desktop/python/pmi/commmodity_short.xlsx', index=False)
commodity_up_df_split.to_excel('/Users/jackyliu/Desktop/python/pmi/commmodity_up.xlsx', index=False)
commodity_down_df_split.to_excel('/Users/jackyliu/Desktop/python/pmi/commmodity_down.xlsx', index=False)

# 2. Output below requires data overrite over the existing file 

updated_index_data.to_excel('/Users/jackyliu/Desktop/python/pmi/pmi_level.xlsx', index=False)
final_comment.to_excel('/Users/jackyliu/Desktop/python/pmi/comments.xlsx', index=False)
sector_ranking_final.to_excel('/Users/jackyliu/Desktop/python/pmi/sector_ranking.xlsx', index=False)